<a href="https://colab.research.google.com/github/hui509/API_Mask_Practice/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C_02_%E8%B3%87%E6%96%99%E5%BA%AB%E4%B8%B2%E6%8E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import sqlite3
import datetime

conn = sqlite3.connect('example.db')
c = conn.cursor()
# 建立pharmacies、masks的資料表並清空
c.execute('CREATE TABLE IF NOT EXISTS pharmacies (city text,counts text,createAt datetime);')
c.execute('DELETE FROM pharmacies;')

c.execute('CREATE TABLE IF NOT EXISTS masks (city text,counts text,createAt datetime);')
c.execute('DELETE FROM masks;')


# 獲得API資料
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)
d = response.text
data = json.loads(d)

# 計算各區域的藥局數量
med_count = {}
for d in data['features']:
    county = d['properties']['address'][0:3]
    if county not in med_count:
        med_count[county] = 1
    else:
        med_count[county] +=1

# 計算各地區剩餘的成人口罩數量
mask_count = {}
for d in data["features"]:
    county = d['properties']['address'][0:3]
    adult = d['properties']['mask_adult']
    if county not in mask_count:
        mask_count[county] = adult
    else:
        mask_count[county] += adult

# 新增pharmacies、masks資料
for key,value in med_count.items():
    t = datetime.datetime.now()
    c.execute(f"INSERT INTO pharmacies VALUES ('{key}',{value},'{t}');")

for key,value in mask_count.items():
    t = datetime.datetime.now()
    c.execute(f"INSERT INTO masks VALUES ('{key}',{value},'{t}');")


# 查詢pharmacies、masks資料
c.execute('SELECT * FROM pharmacies;')
print(c.fetchall())

c.execute('SELECT * FROM masks;')
print(c.fetchall())


conn.commit()
conn.close()